# ECG Full Pipeline — BITalino/OpenSignals (.h5)

1. **outputs/raw/**  
   - `raw/time/*.png` → señal cruda (tiempo)  
   - `raw/freq/*.png` → espectro crudo (FFT)

2. **outputs/filtered/**  
   - `filtered/time/*.png` → señal filtrada (completa) y recorte **20–25 s**  
   - `filtered/freq/*.png` → espectro de la señal filtrada

3. **outputs/pipeline/**  
   - `*_time_filt_peaks.png` → señal filtrada + picos R  
   - `*_time_filt_crop_peaks.png` → recorte 20–25 s + picos R  
   - `*_hr_inst.png` → frecuencia cardiaca instantánea  
   - `*_summary.png` → imagen con métricas (HR media/min/max, SDNN, RMSSD, pNN50, duración, nº de picos)

## 1) Requisitos e instalación (una sola vez)

En el terminal CMD de Windows:

python -m venv .venv
pip install numpy scipy matplotlib h5py

## 2) Imports y utilidades

In [33]:
import os, glob, re, math
import numpy as np
import h5py
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, iirnotch, find_peaks

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)
    return path

## 3) Lectura de archivos `.h5` de BITalino/OpenSignals

OpenSignals guarda datos por **grupos** (cada dispositivo) y dentro un subgrupo `raw` con datasets `channel_#`.  
Extraemos la señal `x`, la frecuencia de muestreo `fs` y construimos el tiempo `t`.

In [34]:
def load_bitalino_h5(path):
    """
    Devuelve dict: canal -> (t, x, meta)
    meta: {"group":..., "fs":..., "n":...}
    """
    out = {}
    with h5py.File(path, 'r') as h5:
        for name in h5.keys():
            grp = h5[name]
            if not isinstance(grp, h5py.Group):
                continue

            # frecuencia de muestreo
            fs = None
            for k in grp.attrs:
                if re.search(r'sampling.?rate|^fs$', str(k), re.I):
                    try:
                        fs = float(grp.attrs[k]); break
                    except:
                        pass

            if 'raw' not in grp:
                continue
            raw = grp['raw']

            channels = [int(k.split('_')[-1]) for k in raw.keys() if k.startswith('channel_')]
            for ch in sorted(channels):
                ds = raw[f'channel_{ch}']
                x = np.squeeze(ds[()]).astype(float)
                N = len(x)
                if N == 0:
                    continue
                t = (np.arange(N)/fs) if (fs and fs>0) else np.arange(N)
                out[ch] = (t, x, {"group": name, "fs": fs, "n": N})
    return out

## 4) Títulos  según el nombre del archivo

In [35]:
def pretty_title_from_filename(base_name):
    b = base_name.lower().replace("_"," ").strip()
    if "reposo" in b: return "Reposo"
    if "respiracion 1" in b or "respiración 1" in b: return "Respiración 1"
    if "respiracion 2" in b or "respiración 2" in b: return "Respiración 2"
    if "respiracion 3" in b or "respiración 3" in b: return "Respiración 3"
    if "primera" in b and "deriv" in b:
        if "ejercicio" in b: return "Primera derivada - Ejercicio"
        return "Primera derivada"
    if "segunda" in b and "deriv" in b:
        if "ejercicio" in b: return "Segunda derivada - Ejercicio"
        return "Segunda derivada"
    if "ejercicio" in b: return "Ejercicio"
    return base_name

## 5) Filtros: notch 60 Hz + pasa-banda 0.5–40 Hz

In [36]:
def bandpass_ecg(x, fs, low=0.5, high=40.0, order=4):
    if not fs or fs <= 0:
        return x
    nyq = 0.5 * fs
    lown = max(1e-6, low/nyq)
    highn = min(0.999, high/nyq)
    if highn <= lown:
        return x
    b, a = butter(order, [lown, highn], btype='band')
    return filtfilt(b, a, x)

def notch_mains(x, fs, f0=60.0, Q=30.0):
    if not fs or fs <= 0 or f0 >= fs/2:
        return x
    b, a = iirnotch(w0=f0/(fs/2), Q=Q)
    return filtfilt(b, a, x)

## 6) FFT (magnitud)

Usamos ventana de **Hann** y normalización por la suma de la ventana.

In [37]:
def magnitude_spectrum(x, fs):
    if not fs or fs <= 0 or len(x) < 2:
        return None, None
    x = x - np.mean(x)
    w = np.hanning(len(x))
    X = np.fft.rfft(x*w)
    f = np.fft.rfftfreq(len(x), d=1.0/fs)
    mag = np.abs(X) / np.sum(w)
    return f, mag


## 7) Detección de picos R y métricas típicas

- Refractario ~250 ms y umbral relativo (percentil 95 × factor).  
- Métricas: **RR**, **FC instantánea**, **HR media/min/max**, **SDNN**, **RMSSD**, **pNN50**.

In [38]:
def detect_r_peaks(x_f, fs):
    if not fs or fs <= 0 or len(x_f) < 5:
        return np.array([], dtype=int)
    min_distance = int(0.25*fs)              # 250 ms
    thr = max(1e-12, np.percentile(x_f,95)*0.35)
    peaks, _ = find_peaks(x_f, distance=min_distance, height=thr)

    # anti-dobles (si dos muy cercanos, deja el mayor)
    if len(peaks) > 1:
        keep = [peaks[0]]
        for i in range(1, len(peaks)):
            if peaks[i] - keep[-1] < int(0.18*fs):
                if x_f[peaks[i]] > x_f[keep[-1]]:
                    keep[-1] = peaks[i]
            else:
                keep.append(peaks[i])
        peaks = np.array(keep, dtype=int)
    return peaks

def hr_metrics_from_peaks(peaks, fs):
    if not fs or fs <= 0 or len(peaks) < 2:
        return {
            "n_R": int(len(peaks)),
            "HR_mean": math.nan, "HR_min": math.nan, "HR_max": math.nan,
            "SDNN": math.nan, "RMSSD": math.nan, "pNN50": math.nan
        }
    rr = np.diff(peaks) / fs     # s
    hr_inst = 60.0 / rr          # lpm
    HR_mean = float(np.mean(hr_inst))
    HR_min  = float(np.min(hr_inst))
    HR_max  = float(np.max(hr_inst))
    SDNN = float(np.std(rr, ddof=1))  # s
    drr = np.diff(rr)
    RMSSD = float(np.sqrt(np.mean(drr**2))) if len(drr)>0 else math.nan
    pNN50 = float(np.mean(np.abs(drr) > 0.05)*100.0) if len(drr)>0 else math.nan
    return {
        "n_R": int(len(peaks)),
        "HR_mean": HR_mean, "HR_min": HR_min, "HR_max": HR_max,
        "SDNN": SDNN, "RMSSD": RMSSD, "pNN50": pNN50
    }


## 8) Helpers de ploteo (guardar figuras)

In [39]:
def save_time_plot(t, x, xlabel, title, out_path):
    plt.figure(figsize=(12,4))
    plt.plot(t, x)
    plt.xlabel(xlabel); plt.ylabel("Amplitud")
    plt.title(title); plt.tight_layout()
    plt.savefig(out_path, dpi=150); plt.close()

def save_time_plot_with_peaks(t, x, peaks, xlabel, title, out_path):
    plt.figure(figsize=(12,4))
    plt.plot(t, x)
    if len(peaks)>0:
        plt.plot(t[peaks], x[peaks], 'o')
    plt.xlabel(xlabel); plt.ylabel("Amplitud (filtrada)")
    plt.title(title); plt.tight_layout()
    plt.savefig(out_path, dpi=150); plt.close()

def save_freq_plot(f, mag, title, out_path, xlim_max=80):
    plt.figure(figsize=(12,4))
    plt.plot(f, mag)
    plt.xlim(0, min(xlim_max, f.max()))
    plt.xlabel("Frecuencia [Hz]"); plt.ylabel("Magnitud (rel.)")
    plt.title(title); plt.tight_layout()
    plt.savefig(out_path, dpi=150); plt.close()

def fmt(x, nd=2):
    return "NA" if (x is None or (isinstance(x, float) and math.isnan(x))) else f"{x:.{nd}f}"

def save_summary_image(metrics_all, metrics_crop, duration_s, n_samples, fs, title, out_path):
    plt.figure(figsize=(7,5))
    plt.axis('off')
    lines = [
        title,
        "",
        f"fs: {fs:.2f} Hz" if fs and fs>0 else "fs: desconocida",
        f"Duración: {duration_s:.2f} s | N muestras: {n_samples}",
        "",
        "— Métricas (toda la señal) —",
        f"R-peaks: {metrics_all['n_R']}",
        f"HR media: {fmt(metrics_all['HR_mean'],1)} lpm",
        f"HR min / max: {fmt(metrics_all['HR_min'],1)} / {fmt(metrics_all['HR_max'],1)} lpm",
        f"SDNN: {fmt(metrics_all['SDNN'],3)} s | RMSSD: {fmt(metrics_all['RMSSD'],3)} s | pNN50: {fmt(metrics_all['pNN50'],1)} %",
        "",
        "— Métricas (20–25 s) —",
        f"R-peaks (crop): {metrics_crop.get('n_R', 'NA')}",
        f"HR media (crop): {fmt(metrics_crop.get('HR_mean', math.nan),1)} lpm"
    ]
    txt = "\n".join(lines)
    plt.text(0.02, 0.98, txt, va='top', fontsize=11, family='monospace')
    plt.tight_layout()
    plt.savefig(out_path, dpi=150, bbox_inches='tight'); plt.close()


## 9) Pipeline completo (RAW, FILTRADOS, PIPELINE)
- Recorre cada archivo y canal para generar:
  - **RAW**: tiempo + FFT crudo.
  - **FILTRADOS**: notch 60 Hz + pasa-banda 0.5–40 Hz; tiempo completo + recorte 20–25 s + FFT.
  - **PIPELINE**: picos R, FC instantánea y **resumen** (imagen).

In [40]:
# === Ajusta aquí la carpeta con tus .h5 ===
ROOT = r"F:\ECG"  # Cambia si es necesario (usa r"ruta\con\backslashes")

# Salidas
OUT_ROOT     = ensure_dir(os.path.join(ROOT, "outputs"))
OUT_RAW_T    = ensure_dir(os.path.join(OUT_ROOT, "raw", "time"))
OUT_RAW_F    = ensure_dir(os.path.join(OUT_ROOT, "raw", "freq"))
OUT_FILT_T   = ensure_dir(os.path.join(OUT_ROOT, "filtered", "time"))
OUT_FILT_F   = ensure_dir(os.path.join(OUT_ROOT, "filtered", "freq"))
OUT_PIPELINE = ensure_dir(os.path.join(OUT_ROOT, "pipeline"))

files = sorted(glob.glob(os.path.join(ROOT, "*.h5")))
print(f"Encontrados {len(files)} archivos .h5 en {ROOT}")

for fpath in files:
    base = os.path.splitext(os.path.basename(fpath))[0]
    pretty = pretty_title_from_filename(base)
    print(f"\n=== {base}  ->  {pretty} ===")

    try:
        channels = load_bitalino_h5(fpath)
        if not channels:
            print("  [!] No se detectaron canales.")
            continue

        for ch, (t, x, meta) in channels.items():
            fs = meta.get("fs", None)
            xlabel = "Tiempo [s]" if (fs and fs>0 and len(t)==len(x)) else "Muestras"

            # ----------- 1) RAW -----------
            save_time_plot(t, x, xlabel, f"{pretty} — {base} — canal_{ch} (crudo)", 
                           os.path.join(OUT_RAW_T, f"{base}_canal{ch}_time_raw.png"))
            f_raw, mag_raw = magnitude_spectrum(x, fs)
            if f_raw is not None:
                save_freq_plot(f_raw, mag_raw, f"{pretty} — {base} — canal_{ch} (FFT crudo)",
                               os.path.join(OUT_RAW_F, f"{base}_canal{ch}_freq_raw.png"))
            else:
                print("  [raw] Espectro omitido (fs desconocida).")

            # ----------- 2) FILTRADO -----------
            x_f = notch_mains(x, fs, 60.0, 30.0)
            x_f = bandpass_ecg(x_f, fs, 0.5, 40.0, 4)

            save_time_plot(t, x_f, xlabel, f"{pretty} — {base} — canal_{ch} (filtrado)",
                           os.path.join(OUT_FILT_T, f"{base}_canal{ch}_time_filt.png"))

            # recorte 20–25 s
            if fs and fs>0:
                i0, i1 = int(20*fs), int(25*fs)
                if i1 <= len(x_f):
                    save_time_plot(t[i0:i1], x_f[i0:i1], "Tiempo [s]",
                                   f"{pretty} — {base} — canal_{ch} (filtrado 20–25 s)",
                                   os.path.join(OUT_FILT_T, f"{base}_canal{ch}_time_filt_crop.png"))
                else:
                    print("  [filt] Señal < 25 s: no hay recorte 20–25 s.")

            f_filt, mag_filt = magnitude_spectrum(x_f, fs)
            if f_filt is not None:
                save_freq_plot(f_filt, mag_filt, f"{pretty} — {base} — canal_{ch} (FFT filtrado)",
                               os.path.join(OUT_FILT_F, f"{base}_canal{ch}_freq_filt.png"))
            else:
                print("  [filt] Espectro filtrado omitido (fs desconocida).")

            # ----------- 3) PIPELINE -----------
            # picos en toda la señal
            peaks = detect_r_peaks(x_f, fs)
            # métricas globales
            duration_s = (len(x)/fs) if (fs and fs>0) else float(len(x))
            n_samples = len(x)
            metrics_all = hr_metrics_from_peaks(peaks, fs)

            # plot R-peaks global
            save_time_plot_with_peaks(
                t, x_f, peaks, xlabel,
                f"{pretty} — {base} — canal_{ch} (picos R)",
                os.path.join(OUT_PIPELINE, f"{base}_canal{ch}_time_filt_peaks.png")
            )

            # HR instantánea
            if fs and fs>0 and len(peaks)>1:
                rr = np.diff(peaks)/fs
                hr_inst = 60.0/rr
                t_rr = (t[peaks][1:] + t[peaks][:-1]) / 2.0
                save_time_plot(
                    t_rr, hr_inst, "Tiempo [s]",
                    f"{pretty} — {base} — canal_{ch} (FC instantánea)",
                    os.path.join(OUT_PIPELINE, f"{base}_canal{ch}_hr_inst.png")
                )

            # recorte 20–25 s con picos
            metrics_crop = {"n_R": "NA", "HR_mean": math.nan}
            if fs and fs>0:
                i0, i1 = int(20*fs), int(25*fs)
                if i1 <= len(x_f):
                    x_fc = x_f[i0:i1]
                    tc   = t[i0:i1]
                    peaks_c = detect_r_peaks(x_fc, fs)
                    m_crop = hr_metrics_from_peaks(peaks_c, fs)
                    metrics_crop = {"n_R": m_crop["n_R"], "HR_mean": m_crop["HR_mean"]}
                    save_time_plot_with_peaks(
                        tc, x_fc, peaks_c, "Tiempo [s]",
                        f"{pretty} — {base} — canal_{ch} (20–25 s, picos R)",
                        os.path.join(OUT_PIPELINE, f"{base}_canal{ch}_time_filt_crop_peaks.png")
                    )

            # resumen (imagen)
            save_summary_image(
                metrics_all, metrics_crop, duration_s, n_samples, fs,
                title=f"{pretty} — {base} — canal_{ch} — Resumen",
                out_path=os.path.join(OUT_PIPELINE, f"{base}_canal{ch}_summary.png")
            )

    except Exception as e:
        print(f"[ERROR] {fpath}: {e}")

print("\nListo. Salidas en:")
print(f"  - RAW tiempo:       {OUT_RAW_T}")
print(f"  - RAW frecuencia:   {OUT_RAW_F}")
print(f"  - FILTRADOS tiempo: {OUT_FILT_T}")
print(f"  - FILTRADOS freq:   {OUT_FILT_F}")
print(f"  - PIPELINE:         {OUT_PIPELINE}")

Encontrados 6 archivos .h5 en F:\ECG

=== PrimeraDerivada  ->  Primera derivada ===

=== RESPIRAECG2  ->  RESPIRAECG2 ===

=== RESPIRAECG3  ->  RESPIRAECG3 ===

=== RESPIRARECG1  ->  RESPIRARECG1 ===

=== ReposoECG_ORIGINAL  ->  Reposo ===

=== SegundaDerivada  ->  Segunda derivada ===

Listo. Salidas en:
  - RAW tiempo:       F:\ECG\outputs\raw\time
  - RAW frecuencia:   F:\ECG\outputs\raw\freq
  - FILTRADOS tiempo: F:\ECG\outputs\filtered\time
  - FILTRADOS freq:   F:\ECG\outputs\filtered\freq
  - PIPELINE:         F:\ECG\outputs\pipeline
